In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def gatherInfoFromURL(url):
    pageRefs = []
    carInfo = {"cars": [], "year": [], "power": [], "carType": []}

    page_response = requests.get(url, timeout=5)
    html_soup = BeautifulSoup(page_response.content, "html.parser")

    pageRefs.append(url)
    for x in html_soup.find("p", class_ = "links").find_all("a"):
        if x["href"] not in pageRefs:
            pageRefs.append(x["href"])

    for page in pageRefs:

        page_response = requests.get(page, timeout=5)
        html_soup = BeautifulSoup(page_response.content, "html.parser")

        carContainers = html_soup.find("section", class_ = "models").find_all("div", class_ = "col-4")

        carsOnPage = [x.find("a").text.strip() for x in carContainers]
        carInfo["cars"].extend(carsOnPage)

        carDetail = [x.find("p") for x in carContainers]

        info = []
        for detail in carDetail:
            for string in detail.strings:
                info.append(string)

        carInfo["year"].extend(info[::2])
        carInfo["power"].extend([x.split(",")[0].strip().split()[0].strip() for x in info[1::2]])
        carInfo["carType"].extend([x.split(",")[1].strip() for x in info[1::2]])

        print(f"Found {len(carsOnPage)} cars on page : {page}")

    return carInfo

In [3]:
carinfo = gatherInfoFromURL(url="http://www.cars-data.com/en/sport-cars.html")

Found 21 cars on page : http://www.cars-data.com/en/sport-cars.html


In [4]:
print(len(carinfo["cars"]))

21


In [5]:
carDataFrame = pd.DataFrame({"Cars": carinfo.get("cars"), "Year": carinfo.get("year"), "Power": carinfo.get("power"), "Car type": carinfo.get("carType"), "Price": carinfo.get("price")})

In [6]:
print(carDataFrame.shape)

(21, 5)


In [7]:
startYear = []
endYear = []

for year in carDataFrame["Year"]:
    splitYear = year.split("-")
    startYear.append(int(splitYear[0].strip()))
    if splitYear[1].strip() == "present":
        endYear.append(2024)
    else:
        endYear.append(int(splitYear[1].strip()))

carDataFrame = carDataFrame.drop(['Year'], axis=1)
carDataFrame["Started"] = startYear
carDataFrame["End year (2024)"] = endYear

print(carDataFrame.head())


                         Cars Power Car type Price  Started  End year (2024)
0  Bugatti Veyron Super Sport  1200    Coupe  None     2010             2015
1         Bugatti Veyron 16.4  1001    Coupe  None     2005             2011
2  Bugatti Veyron Grand Sport  1001    Coupe  None     2009             2015
3       Ferrari SF90 Stradale  1000    Coupe  None     2020             2024
4           Ferrari LaFerrari   963    Coupe  None     2013             2015


In [8]:
header = ["Cars", "Power", "Car type", "Started", "End year (2024)"]
carDataFrame.to_csv("carInfo.csv", columns=header, index=False)